- Symbolic Transformation with gplearn library.
    Parameters found at https://gplearn.readthedocs.io/en/stable/examples.html

### Libraries

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_boston
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path

In [14]:
from gplearn.genetic import SymbolicTransformer
from gplearn.fitness import make_fitness

### Load

In [15]:
boston = load_boston()

# Initializing the dataframe
data = pd.DataFrame(boston.data)

#Adding the feature names to the dataframe
data.columns = boston.feature_names
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [16]:
#Adding target variable to dataframe
data['PRICE'] = boston.target 

In [17]:
# Spliting target variable and independent variables
X = data.drop(['PRICE'], axis = 1)
y = data['PRICE']

In [18]:
# Splitting to training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 4)

In [19]:
# Standarize
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Metric Function

In [20]:
# custom metric
def _mape(y, y_pred, w):
    diffs = np.abs(np.divide((np.maximum(0.001, y) - np.maximum(0.001, y_pred)),np.maximum(0.001, y)))
    
    return 100. * np.average(diffs, weights=w)

mape = make_fitness(_mape, greater_is_better=False)

### Model

In [21]:
function_set = ['add', 'sub', 'mul', 'div',
                'sqrt', 'log', 'abs', 'neg', 'inv',
                'max', 'min']
model = SymbolicTransformer(generations=20, population_size=2000,
                         hall_of_fame=100, n_components=10,
                         function_set=function_set, metric = mape,
                         parsimony_coefficient=0.0005,
                         max_samples=0.9, verbose=1,
                         random_state=0, n_jobs=3)

In [22]:
model.fit(X_train, y_train)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    11.04           106.82        7          58.9013          47.8878      1.23m
   1    15.39          92.3175        8          56.4126          54.0611     32.34s
   2    21.39          85.5198        8          35.0491          36.9155     36.88s
   3    18.88          78.4078       40          33.0389          45.6146     37.52s
   4    21.66          75.9168        9          27.6227          30.0582     39.83s
   5    22.16          72.8139       15          28.0208          24.4646     41.12s
   6    22.38          65.6954       37          26.4276          38.8591     34.72s
   7    28.93          60.7943       41          26.0372          29.0035     35.49s
   8    36.50          51.4358       44          24.7929          25.5808  

SymbolicTransformer(function_set=['add', 'sub', 'mul', 'div', 'sqrt', 'log',
                                  'abs', 'neg', 'inv', 'max', 'min'],
                    max_samples=0.9,
                    metric=<gplearn.fitness._Fitness object at 0x00000181BF76E0D0>,
                    n_jobs=3, parsimony_coefficient=0.0005,
                    population_size=2000, random_state=0, verbose=1)

In [27]:
# Concatenate old and transformed matrix
gp_features = model.transform(boston.data)
new_boston = np.hstack((X, gp_features))

In [28]:
# Saves new matrix
pd.DataFrame(new_boston).to_csv("newBoston.csv")